In [1]:
import domojupyter as dj
import domolibrary_execution.utils.domojupyter as dxdj

auth = await dxdj.generate_domo_auth(
    domo_instance = dxdj.which_environment(),
    domojupyter_fn = dj) 

auth

sdk_domo-community - no password stored in account
🎉 token_auth token retrieved from domo-community ⚙️


DomoTokenAuth(domo_instance='domo-community', url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true', token_name='token_auth', user_id=1893952720, is_valid_token=True)

# Models perform better with examples

In [2]:
import models.model as model

import importlib
importlib.reload(model)

from pprint import pprint

msgs = model.Messages.from_system_prompt('./prompts/python_developer.txt')
msgs

Messages(messages=[Message(role='SYSTEM', content='You are a python developer skilled at translating urls into route functions.  route functions should follow a specific format, see the next few examples.\n\nrequest:  generate a function that GETs the dataflow definition from this url, f"/api/dataprocessing/v1/dataflows/{dataflow_id}" \nresponse:\n\ndef get_dataflow_by_id_sync(\n    auth: dmda.DomoAuth,\n    dataflow_id: int,\n    debug_api: bool = False,\n):\n    url = f"https://{auth.domo_instance}.domo.com/api/dataprocessing/v1/dataflows/{dataflow_id}"     \n    \n    res= client.get_data_sync(\n        auth=auth,\n        url=url,\n        method="GET",\n        debug_api=debug_api\n    )     \n    \n    if not res.is_success:        \n        raise dmde.DomoError(res.response)     \n    \n    return res\n    \nrequest: write a function that PUTs a list of tags for a dataflow using the following url "https://domo-community.domo.com/api/dataprocessing/v1/dataflows/345/tags" and this

In [3]:
import models.model as model

import importlib
importlib.reload(model)

endpoint = model.EndpointHandler._from_creds_account(domo_instance = 'domo-community')


In [4]:

msgs.add_message("""
write a function that POSTs a query to the query api, https://domo-community.domo.com/api/search/v1/query,
the function should receive a query_body of type dictionary which includes the query parameters.
""")
msgs =  endpoint.invoke_message(messages = msgs)

pprint(msgs.messages[-1].content)

('# Function to post a query to the Domo search API\n'
 'def post_search_query_sync(\n'
 '    auth: dmda.DomoAuth,\n'
 '    query_body: dict,\n'
 '    debug_api: bool = False,\n'
 '):\n'
 '    # Construct the URL for the search query API\n'
 '    url = f"https://{auth.domo_instance}.domo.com/api/search/v1/query"\n'
 '    \n'
 '    # Send POST request to the search query API\n'
 '    res = client.get_data_sync(\n'
 '        auth=auth,\n'
 '        url=url,\n'
 '        method="POST",\n'
 '        body=query_body,\n'
 '        debug_api=debug_api\n'
 '    )\n'
 '    \n'
 '    # Raise an error if the request was not successful\n'
 '    if not res.is_success:\n'
 '        raise dmde.DomoError(res.response)\n'
 '    \n'
 '    # Return the response\n'
 '    return res')


## a 'trained model' is not actually trained.  it just has examples included as context

In [5]:
msgs =  endpoint.invoke_message(data = "that's great, now generate a function to GET the tags", messages = msgs)
pprint(msgs.messages[-1].content)

('# Function to get tags from the Domo API\n'
 'def get_tags_sync(\n'
 '    auth: dmda.DomoAuth,\n'
 '    debug_api: bool = False,\n'
 '):\n'
 '    # Construct the URL for the tags API\n'
 '    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/tags"\n'
 '    \n'
 '    # Send GET request to retrieve tags\n'
 '    res = client.get_data_sync(\n'
 '        auth=auth,\n'
 '        url=url,\n'
 '        method="GET",\n'
 '        debug_api=debug_api\n'
 '    )\n'
 '    \n'
 '    # Raise an error if the request was not successful\n'
 '    if not res.is_success:\n'
 '        raise dmde.DomoError(res.response)\n'
 '    \n'
 '    # Return the response containing the tags\n'
 '    return res')


In [6]:
msgs =  endpoint.invoke_message(data = """
i have a filesystem with a file called dataflow.py that should store all these functions we are generating.  
write a function to find if the function exists in the .py file and
if it does replace that function with a new function passed as a string input.""", messages = msgs)

pprint(msgs.messages[-1].content)

("Here's a Python function that checks if a function exists in a 'dataflow.py' "
 'file and replaces it with a new function passed as a string input:\n'
 '\n'
 '```python\n'
 'import re\n'
 'import os\n'
 '\n'
 'def update_function_in_file(new_function_str: str, file_path: str = '
 "'dataflow.py'):\n"
 '    # Extract function name from the new function string\n'
 "    function_name = re.search(r'def\\s+(\\w+)', new_function_str).group(1)\n"
 '    \n'
 '    # Read the existing file content\n'
 "    with open(file_path, 'r') as file:\n"
 '        content = file.read()\n'
 '    \n'
 '    # Check if the function already exists\n'
 '    function_pattern = '
 "re.compile(rf'def\\s+{function_name}\\s*\\(.*?\\):\\s*(?:.*?\\n)+?(?=\\n\\S|\\Z)', "
 're.DOTALL)\n'
 '    if function_pattern.search(content):\n'
 '        # Replace the existing function\n'
 '        updated_content = function_pattern.sub(new_function_str, content)\n'
 '    else:\n'
 '        # Append the new function to the end of t

## if your context is too big, Domo might truncate

In [7]:
len(msgs.messages)

7

In [8]:
import pandas as pd
pd.DataFrame([msg.to_json() for msg in msgs.messages])

,role,content
0,SYSTEM,You are a python developer skilled at translat...
1,USER,\nwrite a function that POSTs a query to the q...
2,SYSTEM,# Function to post a query to the Domo search ...
3,USER,"that's great, now generate a function to GET t..."
4,SYSTEM,# Function to get tags from the Domo API\ndef ...
5,USER,\ni have a filesystem with a file called dataf...
6,SYSTEM,Here's a Python function that checks if a func...


In [9]:
msgs.messages = [msgs.messages[i] for i in [0,3,4]]

In [10]:
pd.DataFrame([msg.to_json() for msg in msgs.messages])

,role,content
0,SYSTEM,You are a python developer skilled at translat...
1,USER,"that's great, now generate a function to GET t..."
2,SYSTEM,# Function to get tags from the Domo API\ndef ...


In [11]:
# msgs =  endpoint.invoke_message(data = """
# i have a filesystem with a file called dataflow.py that should store all these functions we are generating.  
# write a function to find if the function exists in the .py file and
# if it does replace that function with a new function passed as a string input.""", messages = msgs)

# msgs.messages[-1].content

In [12]:
msgs.messages[-1].content

'# Function to get tags from the Domo API\ndef get_tags_sync(\n    auth: dmda.DomoAuth,\n    debug_api: bool = False,\n):\n    # Construct the URL for the tags API\n    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/tags"\n    \n    # Send GET request to retrieve tags\n    res = client.get_data_sync(\n        auth=auth,\n        url=url,\n        method="GET",\n        debug_api=debug_api\n    )\n    \n    # Raise an error if the request was not successful\n    if not res.is_success:\n        raise dmde.DomoError(res.response)\n    \n    # Return the response containing the tags\n    return res'

In [15]:
# import utils

# utils.update_function_in_file('./routes/datacenter.py','post_search_query_sync', msgs.messages[-1].content )